In [74]:
import plotly.express as px
import pandas as pd
import matplotlib.pyplot as plt
import os

Some results (e.g. miconic) are either timing out (due to apptainer and cluster cpus) or running out of memory (possibly due to suboptimal handcrafted policies). Again, we should not worry about this too much. I can just increase the resources.

In [75]:
DOMAINS = [
    "blocksworld",
    "ferry",
    "miconic",
    # "rovers",
    "satellite",
    "transport",
]

In [91]:
# read test logs and write to csv
# f"{domain}_{layer}_{dim}_{pro_blem}_{repeat}"
columns = ["domain", "layer", "dim", "problem", "repeat", "plan_length", "plan_found", "time"]
data = {k: [] for k in columns}
for f in sorted(os.listdir("test_logs")):
    if not f.endswith(".log"):
        continue
    toks = f[:-4].split("_")
    domain = toks[0]
    layer = int(toks[1])
    dim = int(toks[2])
    problem = toks[3] + "_" + toks[4]
    repeat = int(toks[5])
    with open(f"test_logs/{f}") as f:
        content = f.read()
        solved = "Plan generated!" in content
        if solved:
            plan_length = int(content.split("plan_length=")[1].split("\n")[0])
            time = float(content.split("total_time=")[1].split("\n")[0])
        else:
            plan_length = None
            time = None
    data["domain"].append(domain)
    data["layer"].append(layer)
    data["dim"].append(dim)
    data["problem"].append(problem)
    data["repeat"].append(repeat)
    data["plan_length"].append(plan_length)
    data["plan_found"].append(solved)
    data["time"].append(time)

df = pd.DataFrame(data)
df.to_csv("lrnn_results.csv")

In [109]:
def group_repeats(df, others_to_keep=None, lrnn=False):
    if others_to_keep is None:
        others_to_keep = []
    aggr = {
        "time": ["mean", "std"],
        "plan_length": ["mean", "std"],
        "plan_found": "mean",
    }
    if lrnn:
        aggr["layer"] = "first"
        aggr["dim"] = "first"
    group_by = ["domain", "problem"]
    if lrnn:
        group_by += ["layer", "dim"]
    df = df.groupby(group_by).agg(aggr)
    df.columns = df.columns.map("_".join)
    df["all_solved"] = df.plan_found_mean == 1
    df.reset_index(inplace=True)
    return df

In [120]:
groups = ["baseline", "scorpion", "lrnn"]
datas = {}
for group in groups:
    data = pd.read_csv(f"{group}_results.csv")
    is_lrnn = group == "lrnn"
    data = group_repeats(data, lrnn=is_lrnn)
    data["solver"] = group
    if is_lrnn:
        data["solver"] = "lrnn" + "_" + data["layer_first"].astype(str) + "_" + data["dim_first"].astype(str)
    datas[group] = data
display(datas["lrnn"])

,domain,problem,layer,dim,time_mean,time_std,plan_length_mean,plan_length_std,plan_found_mean,layer_first,dim_first,all_solved,solver
0,blocksworld,0_01,1,1,0.392771,0.046180,10.0,0.0,1.0,1,1,True,lrnn_1_1
1,blocksworld,0_02,1,1,0.329650,0.074014,8.0,0.0,1.0,1,1,True,lrnn_1_1
2,blocksworld,0_03,1,1,0.547026,0.003283,20.0,0.0,1.0,1,1,True,lrnn_1_1
3,blocksworld,0_04,1,1,0.532719,0.015227,24.0,0.0,1.0,1,1,True,lrnn_1_1
4,blocksworld,0_05,1,1,0.620619,0.003542,24.0,0.0,1.0,1,1,True,lrnn_1_1
...,...,...,...,...,...,...,...,...,...,...,...,...,...
259,transport,2_22,1,1,NaN,NaN,NaN,NaN,0.0,1,1,False,lrnn_1_1
260,transport,2_23,1,1,NaN,NaN,NaN,NaN,0.0,1,1,False,lrnn_1_1
261,transport,2_24,1,1,NaN,NaN,NaN,NaN,0.0,1,1,False,lrnn_1_1
262,transport,2_26,1,1,NaN,NaN,NaN,NaN,0.0,1,1,False,lrnn_1_1


In [122]:
failure_logs = {domain: [] for domain in DOMAINS}
all_data = pd.concat([data for data in datas.values()])
easy_problems = set(f"0_{i:02d}" for i in range(1, 31))
def plot_domains(metric, log_y=False, easy_only=False):
    for domain in DOMAINS:
        print(domain)
        data = all_data[all_data['domain'] == domain]
        if easy_only:
            data = data[data['problem'].isin(easy_problems)]
        # failures = data[data.plan_found == False]
        # max_bound_failures = data[data.max_bound == False]
        # failures_not_due_to_max_bound = data[(data.plan_found == False) & (data.max_bound == False)]
        # n_failures = len(failures)
        # n_max_bound_failures = len(max_bound_failures)
        # n_failures_not_due_to_max_bound = len(failures_not_due_to_max_bound)
        # print(f"{n_failures=}")
        # for row in failures.itertuples():
        #     # print(row)
        #     failure_logs[domain].append(f"/pfcalcul/work/dchen/code/cvut-colab/slurm/__experiments/baseline_logs/{row.domain}_{row.problem}_r{row.repeat}.log")
        metric_mean = f"{metric}_mean"
        metric_std = f"{metric}_std"
        fig = px.scatter(data, x="problem", y=metric_mean, error_y=metric_std, color="solver", log_y=log_y)
        fig.show()

In [123]:
log_y = 0
easy_only = 0

plot_domains("plan_length", log_y=log_y, easy_only=easy_only)

blocksworld


ferry


miconic


satellite


transport


In [115]:
plot_domains("time")

blocksworld


ferry


miconic


satellite


transport


In [ ]:
domain = "miconic"
for f in failure_logs[domain]:
    print(f)

/pfcalcul/work/dchen/code/cvut-colab/slurm/__experiments/baseline_logs/miconic_0_04_r0.0.log
/pfcalcul/work/dchen/code/cvut-colab/slurm/__experiments/baseline_logs/miconic_0_04_r2.0.log
/pfcalcul/work/dchen/code/cvut-colab/slurm/__experiments/baseline_logs/miconic_0_04_r3.0.log
/pfcalcul/work/dchen/code/cvut-colab/slurm/__experiments/baseline_logs/miconic_0_04_r4.0.log
/pfcalcul/work/dchen/code/cvut-colab/slurm/__experiments/baseline_logs/miconic_0_04_r8.0.log
/pfcalcul/work/dchen/code/cvut-colab/slurm/__experiments/baseline_logs/miconic_0_04_r9.0.log
/pfcalcul/work/dchen/code/cvut-colab/slurm/__experiments/baseline_logs/miconic_0_05_r0.0.log
/pfcalcul/work/dchen/code/cvut-colab/slurm/__experiments/baseline_logs/miconic_0_05_r2.0.log
/pfcalcul/work/dchen/code/cvut-colab/slurm/__experiments/baseline_logs/miconic_0_05_r3.0.log
/pfcalcul/work/dchen/code/cvut-colab/slurm/__experiments/baseline_logs/miconic_0_05_r4.0.log
/pfcalcul/work/dchen/code/cvut-colab/slurm/__experiments/baseline_logs